**_Sum of two numbers_**

In [0]:
x =1
y = 4
print(x+y)

5


**_Sum of List_**

In [0]:
list =[1,2,3,4,5]
print(sum(list))

15


**_Average of List_**

In [0]:
list = [1,2,3,4,5]
print(sum(list)/ len(list))

3.0


**_Tuple_**

In [0]:
tuple = (10,20,30)
print(tuple)

(10, 20, 30)


**_Dictionary_**

In [0]:
my_dict = {"name" : "Vaishnavi" , "age":22}
print(my_dict["name"])
my_dict["age"] = 23
print(my_dict.keys())
print(my_dict.values())

Vaishnavi
dict_keys(['name', 'age'])
dict_values(['Vaishnavi', 23])


**_Set_**

In [0]:
my_set = {1,2,3,4,5}
my_set.add(6)
print(my_set)
my_set.update([2,3,5])
print(my_set)

{1, 2, 3, 4, 5, 6}
{1, 2, 3, 4, 5, 6}


**_Spark Session_**

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PythonDF").getOrCreate()

# Creating DataFrame from list of tuples
data = [(1, "Alice"), (2, "Bob"), (3, "Carol")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
+---+-----+



**_Basic Transformations and Actions on DataFrames_**

In [0]:
# Select specific columns
df.select("name").show()

# Filter rows
df.filter(df["id"] > 1).show()

# Count rows
print(df.count())

# Describe summary statistics (numeric columns)
df.describe().show()

# Add new column with literal value
from pyspark.sql.functions import lit
df = df.withColumn("country", lit("India"))
df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Carol|
+-----+

+---+-----+
| id| name|
+---+-----+
|  2|  Bob|
|  3|Carol|
+---+-----+

3
+-------+---+-----+
|summary| id| name|
+-------+---+-----+
|  count|  3|    3|
|   mean|2.0| null|
| stddev|1.0| null|
|    min|  1|Alice|
|    max|  3|Carol|
+-------+---+-----+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  1|Alice|  India|
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+



**_Working on Sample Sales Data_**

In [0]:
sales_data = [
    ("2024-01-01", "North", "Product A", 10, 200.0),
    ("2024-01-01", "South", "Product B", 5, 300.0),
    ("2024-01-02", "North", "Product A", 20, 400.0),
    ("2024-01-02", "South", "Product B", 10, 600.0),
    ("2024-01-03", "East",  "Product C", 15, 375.0),
]
columns = ["date", "region", "product", "quantity", "revenue"]
sales_df = spark.createDataFrame(sales_data, columns)
sales_df.show()

+----------+------+---------+--------+-------+
|      date|region|  product|quantity|revenue|
+----------+------+---------+--------+-------+
|2024-01-01| North|Product A|      10|  200.0|
|2024-01-01| South|Product B|       5|  300.0|
|2024-01-02| North|Product A|      20|  400.0|
|2024-01-02| South|Product B|      10|  600.0|
|2024-01-03|  East|Product C|      15|  375.0|
+----------+------+---------+--------+-------+



In [0]:
# Product name and total revenue
from pyspark.sql.functions import sum
sales_df.groupBy("product").agg(sum("revenue")).display()

product,sum(revenue)
Product A,600.0
Product B,900.0
Product C,375.0


In [0]:
#Total quantity and revenue by region
from pyspark.sql.functions import sum
sales_df.groupBy("region").agg(sum("quantity"),sum("revenue")).show()

+------+-------------+------------+
|region|sum(quantity)|sum(revenue)|
+------+-------------+------------+
| North|           30|       600.0|
| South|           15|       900.0|
|  East|           15|       375.0|
+------+-------------+------------+



In [0]:
#Average Revenue per product
from pyspark.sql.functions import avg

sales_df.groupBy("product").agg(
    (sum("revenue")/sum("quantity")).alias("avg_revenue")
    ).show()

+---------+-----------+
|  product|avg_revenue|
+---------+-----------+
|Product A|       20.0|
|Product B|       60.0|
|Product C|       25.0|
+---------+-----------+



In [0]:
#Top-Performing
sales_df.groupBy("region").agg(
    sum("revenue").alias("total_revenue")
).orderBy("total_revenue", ascending =False).show(1)


+------+-------------+
|region|total_revenue|
+------+-------------+
| South|        900.0|
+------+-------------+
only showing top 1 row

